# Extracting nuclear morphology and genome organization features 

## Background and aim

The packing of the genome within the nucleus informs transcription and thereby reflects cell state. Therefore, changes in DNA packing information would be a powerful barometer of cell state transitions in important processes such as cancer progression, where there is an unmet need for biomakers. The gold standard for cell abnormalities in cancer is the nuclear grading done by a pathologist using a microscope. High resolution images of DNA as visualised using a fluorescent microscope is a convenient method to measure DNA organization and build a quantitative framework for tracing cell state transitions.

Our samples are obtained from tissue microarray (TMA) slides containing annotated clinical biopsies from cancer patients as well as healthy individuals stained for DNA using Hoechst. In this notebook, we demonstrate how we perform nuclear segmentation and feature extraction.

## Nuclear Segmentation

We segment nuclei using a custom stardist model that was trained using a combination of the 2018 DBS challenge and a few regions from our dataset that were manually annotated in house. 

In [4]:
# import libraries
%load_ext autoreload
import sys
sys.path.append("..")
import os

from pathlib import Path
from glob import glob
import pandas as pd
from tqdm import tqdm

from src.utlis.segmentation_stardist_model import segment_objects_stardist2d
from src.utlis.Run_nuclear_feature_extraction import run_nuclear_chromatin_feat_ext
from src.utlis.summarising_features import summarise_feature_table


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#define paths
path_to_rawimages = '/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Raw_Images/all_cores/'
path_to_output_segmented_images = "/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Segmented_labels/"
path_to_output_ij_rois = "/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Segmented_ij_roi/"

path_to_model = os.path.join(os.path.dirname(os.getcwd()),'models/')
#create output directories if they do not exist
Path(path_to_output_segmented_images).mkdir(parents=True, exist_ok=True)
Path(path_to_output_ij_rois).mkdir(parents=True, exist_ok=True)

In [3]:
#Segmentation
segment_objects_stardist2d(image_dir = path_to_rawimages,
                           output_dir_labels = path_to_output_segmented_images,
                           output_dir_ijroi = path_to_output_ij_rois,
                           use_pretrained = False,
                           model_name ='tissue_nuclear_segmentation',
                           model_dir = path_to_model)

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.507725, nms_thresh=0.3.


100%|██████████| 324/324 [00:15<00:00, 20.84it/s]


## Compute descriptors of spatial chromatin organization 

Following segmentation, we measure features that characterizes the chromatin features using a combination of features from the scikit package as well as custom features such as local curvature and heterochromatin organization. 

In [5]:
## setup paths
path_to_rawimages='/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Raw_Images/all_cores/'
path_to_segmented_images="/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Segmented_labels/"

path_to_output_nuclear_features = "/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/NMCO_features/"
path_to_output_tissue_features = "/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Tissue_Summary/"
#create output directories if they do not exist
Path(path_to_output_nuclear_features).mkdir(parents=True, exist_ok=True)
Path(path_to_output_tissue_features).mkdir(parents=True, exist_ok=True)


In [ ]:
raw_image_dirs = sorted(glob(path_to_rawimages + "*.tif"))
seg_image_dirs = sorted(glob(path_to_segmented_images + "*.tif"))

### Extract single nuclear features for all images and summarise at tissue level
Tissue_summary = pd.DataFrame()

for i in tqdm(range(len(raw_image_dirs))):
    
    feature = run_nuclear_chromatin_feat_ext(raw_image_dirs[i],seg_image_dirs[i],path_to_output_nuclear_features)
    
    feature = feature.drop(['Image','label'], axis=1)
    feature = feature.replace('NA',0, regex=True)
    feature = feature.replace('NaN',0, regex=True)
    
    temp = summarise_feature_table(feature)
    temp['Image'] = raw_image_dirs[i].rsplit('/', 1)[-1][:-4]
    Tissue_summary=pd.concat([Tissue_summary, temp], ignore_index=True, axis=0)
    
    del feature, temp

Tissue_summary.to_csv(path_to_output_tissue_features+"NMCO_tissue_summary.csv")

  0%|          | 0/77 [00:00<?, ?it/s]

  0%|          | 0/3561 [00:00<?, ?it/s]

  1%|▏         | 1/77 [04:01<5:05:29, 241.17s/it]

  0%|          | 0/3807 [00:00<?, ?it/s]

  3%|▎         | 2/77 [08:19<5:13:46, 251.02s/it]

...

  0%|          | 0/4736 [00:00<?, ?it/s]

 65%|██████▍   | 50/77 [4:41:08<2:03:57, 275.45s/it]

  0%|          | 0/4242 [00:00<?, ?it/s]

 66%|██████▌   | 51/77 [4:45:13<1:55:30, 266.54s/it]

  0%|          | 0/7350 [00:00<?, ?it/s]